# Emoji Predictor


In [2]:
!pip install emoji

     |████████████████████████████████| 51kB 3.1MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42176 sha256=3bceb5151243f1b15c310456863353a2bcd054f70fc10e8793c9250edd191034
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


Importing Libraries


In [0]:
import emoji
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import *
from keras.utils import to_categorical

# Data Processing

In [0]:
XT = pd.read_csv('xTrain.csv',header=None)
Xt = pd.read_csv('xTest.csv',header=None)
YT = pd.read_csv('yTrain.csv',header=None)
Yt = pd.read_csv('yTest.csv',header=None)

In [0]:
trainX = []
for i in range(0,len(XT)):
    for j in XT[0][i].split("' '"):
        trainX.append(j)
for i in range(0,len(trainX)):
    if(trainX[i][0]==' '):
        trainX[i] = trainX[i][1:]
    if(trainX[i][0]=="'"):
        trainX[i] = trainX[i][1:]
    if(trainX[i][-1]=="'"):
        trainX[i] = trainX[i][:-1]
    if(trainX[i][-1]==' '):
        trainX[i] = trainX[i][:-1]
    if(trainX[i][-1]=="'"):
        trainX[i] = trainX[i][:-1]
trainX[-1] = trainX[-1][:-2]

In [0]:
trainY = []
for i in range(len(YT)):
    x = YT[0][i]
    x = x.strip(' ').split(' ')
    for i in x:
        trainY.append(i)

In [0]:
testX = []
for i in range(0,len(Xt)):
    for j in Xt[0][i].split("' '"):
        testX.append(j)
for i in range(0,len(testX)):
    if(testX[i][0]==' '):
        testX[i] = testX[i][1:]
    if(testX[i][0]=="'"):
        testX[i] = testX[i][1:]
    if(testX[i][-1]=="'"):
        testX[i] = testX[i][:-1]
    if(testX[i][-2:]=='\\t'):
        testX[i] = testX[i][:-2]
    if(testX[i][-1]==' '):
        testX[i] = testX[i][:-1]
    if(testX[i][-1]=="'"):
        testX[i] = testX[i][:-1]

In [0]:
testY = []
for i in range(len(Yt)):
    x = Yt[0][i]
    x = x.strip(' ').split(' ')
    for i in x:
        testY.append(i)

In [0]:
X_train = []
for i in trainX:
  x = i.split(' ')
  X_train.append(x)

In [0]:
X_test = []
for i in testX:
  x = i.split(' ')
  X_test.append(x)

In [16]:
print(len(trainX),len(trainY))
print(len(testX),len(testY))

132 132
56 56


In [0]:
emoji_dictionary = {
                    "0": "\u2764\uFE0F",
                    "1": ":baseball:",
                    "2": ":grinning_face_with_big_eyes:",
                    "3": ":disappointed_face:",
                    "4": ":fork_and_knife:"
                   }

In [0]:
f = open('glove.6B.50d.txt',encoding='utf-8')

In [19]:
embeddings_index = {}
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float')
    print(word,coefs)
    embeddings_index[word] = coefs
f.close()

Streaming output truncated to the last 5000 lines.
  0.12935   -0.66863   -0.45017    1.0072    -0.026043  -0.33214
  1.1586    -0.15123   -0.65852    0.44453   -0.16374   -0.35472
 -0.55359    0.80902    0.49164   -0.32145   -0.42331   -0.075476
 -0.46313    0.5845     0.37874    0.18467   -0.24364    0.2784
 -0.19259    0.38496    0.29154    0.057476  -0.0016944 -0.0051214
 -0.58271    0.25844  ]
olin [-0.63245   0.26131   0.43664   0.49697  -0.21692   0.73617  -0.36252
 -0.89369  -0.020895 -0.66061  -0.13358   0.64367  -0.20143  -0.17064
 -1.057     0.17404   0.374     0.65475  -0.34919   0.55974   0.57797
  0.65303   0.18927  -0.78579   0.89845  -0.40323   0.033009 -0.52115
 -0.88814  -0.6245   -0.91236   0.33773   0.77277   0.027251 -0.72174
 -0.70375  -0.3921    0.61899   0.65113   0.46075   0.50927  -0.1006
  0.050634  0.061371  0.41397   0.21431  -0.50479  -0.16706  -0.56072
  0.81055 ]
bane [ 0.41033   -0.32262    0.1257    -0.21923   -0.067596   0.36184
  0.27348    0.19181  

In [20]:
emb_dim = embeddings_index['a'].shape[0]
print(emb_dim)

50


In [0]:
Y_train = to_categorical(trainY,num_classes=5)
Y_test = to_categorical(testY,num_classes=5)

In [61]:
print(Y_train.shape)
print(Y_train[0])

(132, 5)
[0. 0. 0. 1. 0.]


In [0]:
def embedding_output(X):
  maxLen = 10
  embedding_out = np.zeros((len(X),maxLen,emb_dim))
  for ix in range(len(X)):
    X[ix] = X[ix].split()
    for ij in range(len(X[ix])):
      try:
        embedding_out[ix][ij] = embeddings_index[X[ix][ij].lower()]
      except:
        embedding_out[ix][ij] = np.zeros((50,))
  return embedding_out

In [23]:
print(X_train[0])
print(len(X_train[0]))

['never', 'talk', 'to', 'me', 'again']
5


In [0]:
embedding_matrix_train = embedding_output(trainX)
embedding_matrix_test = embedding_output(testX)

In [40]:
embedding_matrix_train.shape,embedding_matrix_test.shape

((132, 10, 50), (56, 10, 50))

# Creating a LSTM Model

In [65]:
model = Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64,return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_2 (Activation)    (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
__________________________________________________

In [66]:
hist = model.fit(embedding_matrix_train,Y_train,epochs=100,batch_size=64,shuffle=True,validation_split=0.2)

Train on 105 samples, validate on 27 samples
Epoch 1/100
105/105 [==============================] - 1s 8ms/step - loss: 1.6328 - accuracy: 0.1524 - val_loss: 1.5884 - val_accuracy: 0.2963
Epoch 2/100
105/105 [==============================] - 0s 1ms/step - loss: 1.5862 - accuracy: 0.2476 - val_loss: 1.5901 - val_accuracy: 0.2222
Epoch 3/100
105/105 [==============================] - 0s 1ms/step - loss: 1.5677 - accuracy: 0.3333 - val_loss: 1.5963 - val_accuracy: 0.2222
Epoch 4/100
105/105 [==============================] - 0s 1ms/step - loss: 1.5326 - accuracy: 0.3333 - val_loss: 1.6071 - val_accuracy: 0.2593
Epoch 5/100
105/105 [==============================] - 0s 1ms/step - loss: 1.5104 - accuracy: 0.3143 - val_loss: 1.6255 - val_accuracy: 0.2593
Epoch 6/100
105/105 [==============================] - 0s 1ms/step - loss: 1.4910 - accuracy: 0.3143 - val_loss: 1.6454 - val_accuracy: 0.2222
Epoch 7/100
105/105 [==============================] - 0s 1ms/step - loss: 1.4699 - accuracy: 0.3

In [67]:
pred = model.predict_classes(embedding_matrix_test)
print(pred)

[4 3 2 0 2 2 3 2 4 2 1 2 0 3 1 3 2 0 3 2 3 3 4 2 3 3 1 0 1 2 0 1 3 2 2 1 2
 4 4 2 1 0 0 2 2 2 3 2 3 3 3 0 3 2 2 3]


In [68]:
model.evaluate(embedding_matrix_test,Y_test)

56/56 [==============================] - 0s 322us/step


[1.1967622893197196, 0.7678571343421936]

# Predicting Emojis

In [74]:
for i in range(0,len(pred)):
  print(' '.join(testX[i]))
  print(emoji.emojize(emoji_dictionary[str(np.argmax(Y_test[i]))]))
  print(emoji.emojize(emoji_dictionary[str(pred[i])]))

I want to eat
🍴
🍴
he did not answer
😞
😞
he got a very nice raise
😃
😃
she got me a nice present
😃
❤️
ha ha ha it was so funny
😃
😃
he is a good friend
😃
😃
I am upset
😞
😞
We had such a lovely dinner tonight
😃
😃
where is the food
🍴
🍴
Stop making this joke ha ha ha
😃
😃
where is the ball
⚾
⚾
work is hard
😞
😃
This girl is messing with me
😞
❤️
are you serious
😞
😞
Let us go play baseball
⚾
⚾
This stupid grader is not working
😞
😞
work is horrible
😞
😃
Congratulation for having a baby
😃
❤️
stop pissing me off
😞
😞
any suggestions for dinner
🍴
😃
I love taking breaks
❤️
😞
you brighten my day
😃
😞
I boiled rice
🍴
🍴
she is a bully
😞
😃
Why are you feeling bad
😞
😞
I am upset
😞
😞
give me the ball
⚾
⚾
My grandmother is the love of my life
❤️
❤️
enjoy your game
⚾
⚾
valentine day is near
😃
😃
I miss you so much
❤️
❤️
throw the ball
⚾
⚾
My life is so boring
😞
😞
she said yes
😃
😃
will you be my valentine
😃
😃
he can pitch really well
⚾
⚾
dance with me
😃
😃
I am hungry
🍴
🍴
See you at the restaurant
🍴
🍴
I like to lau